In [1]:
#Import required liibraries
import IPython
import matplotlib.pyplot as plt
import IPython.display as Disp
from ipywidgets import widgets
import numpy as np
import cv2
import plotly.graph_objects as go
from ipywidgets import interact, interactive
import glob

In [2]:
button = widgets.Button(description="Start Iterations")
output_box = widgets.Output()
display(output_box)

Output()

In [3]:
print('Tracking Dashboard')

Tracking Dashboard


In [4]:
folders = glob.glob('Videos/*')
def loadfiles(folder):
    files = sorted(glob.glob('Videos' + folder + '/*.mp4'))
    return files

def video(files):
    huh = 8
for i in range(len(folders)):
    folders[i] = folders[i].split('/')[1]
folderpaths = interactive(loadfiles, folder=folders)
display(folderpaths)

interactive(children=(Dropdown(description='folder', options=('080819',), value='080819'), Output()), _dom_cla…

In [7]:
%matplotlib widget

In [4]:
#Initalize Trackers


OPENCV_OBJECT_TRACKERS = {
    "csrt": cv2.TrackerCSRT_create,
    "kcf": cv2.TrackerKCF_create,
    "boosting": cv2.TrackerBoosting_create,
    "mil": cv2.TrackerMIL_create,
    "tld": cv2.TrackerTLD_create,
    "medianflow": cv2.TrackerMedianFlow_create,
    "mosse": cv2.TrackerMOSSE_create
}

trackers = cv2.MultiTracker_create()



In [5]:
#Define the mouse input parameters
class bbox_select():
    #%matplotlib notebook
    
    def __init__(self,im):
        self.tissues = 2
        self.im = im
        self.clicks = 0
        self.selected_points = []
        for i in range(self.tissues):
            self.selected_points.append([])
        self.fig,ax = plt.subplots()
        self.img = ax.imshow(self.im.copy())
        self.ka = self.fig.canvas.mpl_connect('button_press_event', self.onclick)
        disconnect_button = widgets.Button(description="Selected Posts")
        Disp.display(disconnect_button)
        disconnect_button.on_click(self.disconnect_mpl)

    def poly_img(self,img,pts):
        for point in pts:
            temp = np.array(point, np.int32)
            point = temp.reshape((-1,1,2))
            cv2.polylines(img,[point],True,(np.random.randint(0,255),np.random.randint(0,255),np.random.randint(0,255)),7)
            #cv2.rectangle(img, point[0], point[1])
        return img

    def onclick(self, event):
        for i in range(self.tissues):
            if len(self.selected_points[i])<2:   
                self.selected_points[i].append([event.xdata,event.ydata])
                break
                
        if len(self.selected_points[0])>1:
            self.fig
            self.img.set_data(self.poly_img(self.im.copy(),self.selected_points))

        
    def disconnect_mpl(self,_):
        self.fig.canvas.mpl_disconnect(self.ka)

In [6]:
#Import the video and call the mouse input function
videostream = cv2.VideoCapture('Videos/080819/video.mp4')
global images
images = videostream.read()[1]
bs = bbox_select(images)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Button(description='Selected Posts', style=ButtonStyle())

In [7]:
def modifybox(badpoints):
    boxes = badpoints

    for i in range(len(badpoints)):
        count = 0
        for (xpoint, ypoint) in badpoints[i]:
            if (count % 2) == 1:
                boxes[i] = (xprev, yprev, xpoint - xprev, ypoint - yprev)
            else:
                xprev = xpoint
                yprev = ypoint
            count+=1
    return boxes
#box = modifybox(bs.selected_points)

In [8]:

mybutton = widgets.Button(description="nothing")

def on_button_clicked(b):
    box = modifybox(bs.selected_points)
    
    for i in range(len(box)):
        tracker = OPENCV_OBJECT_TRACKERS['csrt']()
        trackers.add(tracker, images, box[i])
    
    count = 0
    lists = []
    xox = []

    fig = go.Figure()
    trace = fig.add_trace(go.Scatter(x=xox,y=lists)) 
    while True:
        #count = count + 1
        print(count)
        if count >= 100:
            break
        image = videostream.read()[1]
        if image is None:
            break
        posts = trackers.update(image)[1]
        postcords = []

        for post in posts:
            print('yeye')
            #Used float for more acuracy but rectangle needs int
            (x, y, w, h) = [float(i) for i in post]
            (rx, ry, rw, rh) = [int(i) for i in (x, y, w, h)]
            cv2.rectangle(image, (rx, ry), (rx + rw, ry + rh), (0, 255, 0), 2)
            #Populate list for centroid tracking
            postcords.append((x, y, x + w, y + h))  


        postio = {}

        for (i, (x, y, x2, y2)) in enumerate(postcords):
                print('here')
                centroidX = int((x + x2) / 2.0)
                centroidY = int((y + y2) / 2.0)
                postio[i] = (centroidX, centroidY)
        #print(postio)

        for (objectID, centroid) in postio.items():
            #print(centroid)
            #print('there')
            #text = "{}".format(objectID)

            #cv2.putText(image, text, (centroid[0] - 10, centroid[1] - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)
            #cv2.circle(image, (centroid[0], centroid[1]), 4, (0, 255, 0), -1)
            #plt.imshow(image)

            if (objectID % 2) == 0:
                    #Save the x position of the even post
                    evenX = centroid[0]
                    #Save the y position of the even post
                    evenY = centroid[1]
                    evenID = objectID
                    #If objectID is odd it has a pair so do stuff
            elif (objectID - 1) == evenID:
                    #Calculate tissue number based on object ID
                    reltissueID = int((objectID - 1)/2)
                    #Save the x position of the odd post
                    oddX = centroid[0]
                    #Save the y position of the odd post
                    oddY = centroid[1]

                    disp = np.sqrt(((oddX - evenX)**2) + ((oddY - evenY)**2))
                    xox.append(count)
                    count = count + 1
                    print(count)
                    lists.append(disp)
            trace.data[0].x = xox
            trace.data[0].y = lists
        #trace = fig.add_trace(go.Scatter(x=xox,y=lists)) 
            with output_box:
                output_box.clear_output(wait=True)
                display(fig)#= fig.add_trace(go.Scatter(x=xox,y=lists)) 
    
    return box


display(button)
button.on_click(on_button_clicked)

SyntaxError: 'return' outside function (<ipython-input-8-b62039deb8a6>, line 82)

NameError: name 'box' is not defined

interactive(children=(Checkbox(value=True, description='x'), Output()), _dom_classes=('widget-interact',))

In [11]:
out = widgets.Output()

@out.capture(clear_output=True, wait=True)
def function_with_captured_output():
    with out:
        print('This goes into the output widget')
    with out:
        print('other')
    

function_with_captured_output()
out

Output()